In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
saife245_english_premier_league_path = kagglehub.dataset_download('saife245/english-premier-league')

print('Data source import complete.')


100%|██████████| 1.12M/1.12M [00:00<00:00, 29.9MB/s]

Extracting files...
Data source import complete.


### Importing the dataset

import pandas as pd
This imports the Pandas library, It allows us to work efficiently with data tables using DataFrame and Series structures.

import matplotlib.pyplot as plt, It provides functions to create various types of charts like line plots, bar charts, scatter plots, etc.

import numpy as np, It is mainly used for numerical computations and working with arrays and matrices.

import seaborn as sns, It offers advanced plotting functions for visualizing relationships and patterns in data with better default aesthetics.

from datetime import datetime as dt, allowing us to parse, format, and manipulate date and time values. It is aliased as dt for convenience.

import itertools, provides functions that create iterators for efficient looping. It's useful for creating combinations, permutations, and other iterator-based tasks.

%matplotlib inline
This is a Jupyter/Colab-specific magic command. It ensures that all Matplotlib plots are displayed directly within the notebook cells instead of in separate windows.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools

%matplotlib inline

This code defines the file path for the datasets, lists the seasons to be included, and then reads each season's data from a CSV file into a list of pandas DataFrames.

In [ ]:
# Define the folder path
folder = saife245_english_premier_league_path + '/Datasets/'

# Define the file names for the seasons
seasons = [
    '2000-01', '2001-02', '2002-03', '2003-04', '2004-05',
    '2005-06', '2006-07', '2007-08', '2008-09', '2009-10',
    '2010-11', '2011-12', '2012-13', '2013-14', '2014-15',
    '2015-16', '2016-17', '2017-18'
]

# Read all CSV files into a list of DataFrames
raw_data = [pd.read_csv(f'{folder}{season}.csv') for season in seasons]

In [ ]:
# Columns we want to keep
columns_req = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

# Extract only the required columns from each season's DataFrame
playing_statistics = [df[columns_req] for df in raw_data]


**GOALS SCORED AND CONCEDED AT THE END OF MATCHWEEK, ARRANGED BY TEAMS AND MATCHWEEK**

In [ ]:
# Function to calculate cumulative goals scored per matchweek
def get_goals_scored(playing_stat):
    # Initialize a dictionary where each team will have a list of goals scored per match
    teams = {team: [] for team in playing_stat['HomeTeam'].unique()}

    # Loop over each match in the dataset
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']  # Full Time Home Goals scored by Home Team
        ATGS = playing_stat.iloc[i]['FTAG']  # Full Time Away Goals scored by Away Team

        # Append goals scored to respective team lists
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)

    # Convert the dictionary into a DataFrame with weeks as columns (1 to 38), teams as rows
    GS = pd.DataFrame(data=teams, index=range(1, 39)).T

    # Initialize matchweek 0 as 0 goals scored
    GS[0] = 0

    # Compute cumulative goals scored by each team up to each matchweek
    for i in range(2, 39):
        GS[i] = GS[i] + GS[i - 1]

    return GS


# Function to calculate cumulative goals conceded per matchweek
def get_goals_conceded(playing_stat):
    # Initialize a dictionary for each team's goals conceded per match
    teams = {team: [] for team in playing_stat['HomeTeam'].unique()}

    # Loop over each match
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']  # Home team goals = goals conceded by away team
        HTGC = playing_stat.iloc[i]['FTAG']  # Away team goals = goals conceded by home team

        # Append goals conceded to respective team lists
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)

    # Convert dictionary into a DataFrame with weeks as columns (1 to 38), teams as rows
    GC = pd.DataFrame(data=teams, index=range(1, 39)).T

    # Initialize matchweek 0 as 0 goals conceded
    GC[0] = 0

    # Compute cumulative goals conceded by each team up to each matchweek
    for i in range(2, 39):
        GC[i] = GC[i] + GC[i - 1]

    return GC


# Main function to append cumulative stats to the playing_stat DataFrame
def get_gss(playing_stat):
    # Make a copy of the DataFrame to avoid SettingWithCopyWarning
    playing_stat = playing_stat.copy()

    # Get cumulative goals conceded and scored
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)

    # Initialize lists to store computed stats for each row/match
    HTGS, ATGS, HTGC, ATGC = [], [], [], []
    matchweek = 0  # Track which matchweek we are in

    # Loop over each match in the dataset
    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam  # Get home team name
        at = playing_stat.iloc[i].AwayTeam  # Get away team name

        # Append cumulative stats up to current matchweek
        HTGS.append(GS.loc[ht][matchweek])
        ATGS.append(GS.loc[at][matchweek])
        HTGC.append(GC.loc[ht][matchweek])
        ATGC.append(GC.loc[at][matchweek])

        # Update matchweek every 10 matches (10 matches per week in EPL)
        if (i + 1) % 10 == 0:
            matchweek += 1

    # Add new columns to the dataset for each stat
    playing_stat['HTGS'] = HTGS  # Home Team Goals Scored so far
    playing_stat['ATGS'] = ATGS  # Away Team Goals Scored so far
    playing_stat['HTGC'] = HTGC  # Home Team Goals Conceded so far
    playing_stat['ATGC'] = ATGC  # Away Team Goals Conceded so far

    return playing_stat


# Apply the get_gss function to a list of playing statistics DataFrames
playing_statistics = [get_gss(df) for df in playing_statistics]

**GET RESPECTIVE POINTS**

In [ ]:
# Function to convert match result to points
def get_match_points(result):
    # Win gives 3 points
    if result == 'W':
        return 3
    # Draw gives 1 point
    elif result == 'D':
        return 1
    # Loss gives 0 points
    else:
        return 0


# Function to generate raw match results (W/D/L) for each team
def get_match_results(playing_stat):
    # Initialize a dictionary for each team with an empty list
    teams = {team: [] for team in playing_stat['HomeTeam'].unique()}

    # Loop over each row/match to populate results
    for i in range(len(playing_stat)):
        result = playing_stat.iloc[i].FTR  # Match result: 'H', 'A', or 'D'

        home = playing_stat.iloc[i].HomeTeam
        away = playing_stat.iloc[i].AwayTeam

        # Append match result for each team
        if result == 'H':
            teams[home].append('W')   # Home win
            teams[away].append('L')   # Away loss
        elif result == 'A':
            teams[away].append('W')   # Away win
            teams[home].append('L')   # Home loss
        else:
            teams[home].append('D')   # Draw
            teams[away].append('D')   # Draw

    # Convert to DataFrame (weeks = columns 1-38, teams = rows)
    match_results_df = pd.DataFrame(data=teams, index=range(1, 39)).T
    return match_results_df


# Function to convert raw match results to cumulative points
def get_cumulative_points(match_results_df):
    # Apply point conversion function to each cell
    match_points = match_results_df.map(get_match_points)

    # Initialize week 0 as 0 points
    match_points[0] = 0

    # Compute cumulative points for each team up to each week
    for i in range(2, 39):
        match_points[i] = match_points[i] + match_points[i - 1]

    return match_points


# Main function to append team cumulative points to each row in the dataset
def get_team_points(playing_stat):
    # Make a copy of the DataFrame to avoid SettingWithCopyWarning
    playing_stat = playing_stat.copy()

    # Step 1: Get W/D/L for each team
    match_results = get_match_results(playing_stat)

    # Step 2: Get cumulative points for each team
    cumulative_points = get_cumulative_points(match_results)

    # Step 3: Assign cumulative points to each match row
    HTP, ATP = [], []
    matchweek = 0

    for i in range(len(playing_stat)):
        home = playing_stat.iloc[i].HomeTeam
        away = playing_stat.iloc[i].AwayTeam

        # Fetch cumulative points up to the current matchweek
        HTP.append(cumulative_points.loc[home][matchweek])
        ATP.append(cumulative_points.loc[away][matchweek])

        # Update matchweek every 10 matches
        if (i + 1) % 10 == 0:
            matchweek += 1

    # Add points columns to the DataFrame
    playing_stat['HTP'] = HTP  # Home Team Points
    playing_stat['ATP'] = ATP  # Away Team Points

    return playing_stat

**GET TEAM FORM:**

In [ ]:
# Function to compute rolling match results (form) for each team
def get_form_results(playing_stat):
    match_results = get_match_results(playing_stat)
    form_df = match_results.copy()

    # For each team, concatenate past match results to form a string
    for week in range(1, 39):
        for team in form_df.index:
            if week < 6:
                # Not enough matches played yet — assign 'M' for missing
                form_df.at[team, week] = 'M' * 5
            else:
                # Form is composed of the last 5 match results before this week
                form_df.at[team, week] = ''.join(match_results.loc[team][week - 5:week])

    return form_df

# Function to add form columns HM1–HM5 and AM1–AM5 to the dataset
def get_form(playing_stat):
    form_df = get_form_results(playing_stat)

    # Initialize lists to hold form info per match
    HM1, HM2, HM3, HM4, HM5 = [], [], [], [], []
    AM1, AM2, AM3, AM4, AM5 = [], [], [], [], []

    matchweek = 0

    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam

        # Assign 'M' if form not yet available
        if matchweek < 1:
            HM1.append('M'); HM2.append('M'); HM3.append('M'); HM4.append('M'); HM5.append('M')
            AM1.append('M'); AM2.append('M'); AM3.append('M'); AM4.append('M'); AM5.append('M')
        else:
            home_form = form_df.loc[ht][matchweek]
            away_form = form_df.loc[at][matchweek]

            HM1.append(home_form[4])
            HM2.append(home_form[3])
            HM3.append(home_form[2])
            HM4.append(home_form[1])
            HM5.append(home_form[0])

            AM1.append(away_form[4])
            AM2.append(away_form[3])
            AM3.append(away_form[2])
            AM4.append(away_form[1])
            AM5.append(away_form[0])

        # Every 10 matches = new matchweek
        if (i + 1) % 10 == 0:
            matchweek += 1

    # Add columns to DataFrame
    playing_stat['HM1'] = HM1; playing_stat['HM2'] = HM2; playing_stat['HM3'] = HM3
    playing_stat['HM4'] = HM4; playing_stat['HM5'] = HM5

    playing_stat['AM1'] = AM1; playing_stat['AM2'] = AM2; playing_stat['AM3'] = AM3
    playing_stat['AM4'] = AM4; playing_stat['AM5'] = AM5

    return playing_stat
# Apply the get_form function to all playing_stat datasets
playing_statistics = [get_form(df) for df in playing_statistics]



In [ ]:
# Apply the get_gss function to a list of playing statistics DataFrames
playing_statistics = [get_gss(df) for df in playing_statistics]

# Apply the get_team_points function to all playing_stat datasets
playing_statistics = [get_team_points(df) for df in playing_statistics]

# Desired column order
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
        'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP',
        'HM1', 'HM2', 'HM3', 'HM4', 'HM5',
        'AM1', 'AM2', 'AM3', 'AM4', 'AM5']

# Apply the column rearrangement to each playing_statistics_i
playing_statistics = [df[cols] for df in playing_statistics]

**Get MatchWeek:**

In [ ]:
# Function to add MatchWeek column (MW) based on number of matches
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):  # EPL season has 380 matches
        MatchWeek.append(j)
        if ((i + 1) % 10) == 0:  # Every 10 matches = 1 week
            j += 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

# Apply get_mw to all season DataFrames
playing_statistics = [get_mw(df) for df in playing_statistics]


**FINAL DATAFRAME**

In [ ]:
# Combine all playing_statistics DataFrames into one
playing_stat = pd.concat(
    playing_statistics,
    ignore_index=True
)

# Helper function to assign points from result character
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    elif result == 'L':
        return 0
    else:
        return 0  # Handle unexpected values

# Calculates total form points from a string like 'WDWLW'
def get_form_points(string):
    return sum(get_points(letter) for letter in string)

# Create home and away form strings
home_matches = [f'HM{i}' for i in range(1, 6)]
away_matches = [f'AM{i}' for i in range(1, 6)]

playing_stat['HTFormPtsStr'] = playing_stat[home_matches].agg(''.join, axis=1)
playing_stat['ATFormPtsStr'] = playing_stat[away_matches].agg(''.join, axis=1)

# Apply form points calculation
playing_stat['HTFormPts'] = playing_stat['HTFormPtsStr'].apply(get_form_points)
playing_stat['ATFormPts'] = playing_stat['ATFormPtsStr'].apply(get_form_points)

# Streak helper functions
def get_streak(string, pattern):
    return int(string.endswith(pattern))

def is_exact_streak(string, pattern):
    return int(string == pattern)

# Generate win/loss streak flags for Home and Away teams
for prefix in ['HT', 'AT']:
    form_str_col = f'{prefix}FormPtsStr'

    playing_stat[f'{prefix}WinStreak3'] = playing_stat[form_str_col].apply(lambda x: get_streak(x, 'WWW'))
    playing_stat[f'{prefix}WinStreak5'] = playing_stat[form_str_col].apply(lambda x: is_exact_streak(x, 'WWWWW'))
    playing_stat[f'{prefix}LossStreak3'] = playing_stat[form_str_col].apply(lambda x: get_streak(x, 'LLL'))
    playing_stat[f'{prefix}LossStreak5'] = playing_stat[form_str_col].apply(lambda x: is_exact_streak(x, 'LLLLL'))

# Check column names at the end
print(playing_stat.columns)

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'MW', 'HTFormPtsStr', 'ATFormPtsStr',
       'HTFormPts', 'ATFormPts', 'HTWinStreak3', 'HTWinStreak5',
       'HTLossStreak3', 'HTLossStreak5', 'ATWinStreak3', 'ATWinStreak5',
       'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')
